# Generate a Question on the document

In [21]:
import data_formats
import entities

import copy
import json
import os
import pickle
import pprint
import random

StorageEnvVarName = 'STORAGE_ROOT'
assert StorageEnvVarName in os.environ
StorageEnvVar= os.environ[StorageEnvVarName]
print(f'StorageEnvVar:{StorageEnvVar}')

StorageEnvVar:/home/minguzzi/repo/quiz_generator/storage


In [2]:
document = entities.Document()

documentFile= os.path.join( StorageEnvVar,'document.pickle' )
with open(documentFile, 'rb') as file:
    document = pickle.load(file)

# Now `data` holds the original data structure
document.numArticles

61

In [4]:
if document.numArticles>0 :
    indxArticle = random.randint(0, document.numArticles-1)
    print(indxArticle)
    article = document.article(indxArticle)
    print(article.title)
    print(article.text)
    

2
Causa di Morte
1. A norma dell'art. 103 T.U. Leggi Sanitarie, R.D. 27.7.1934 n. 1265, i medici debbono per ogni caso di morte di persona da loro assistita denunciare al Sindaco la malattia che, a loro giudizio, ne sarebbe stata la causa. 

2. Nel caso di morte per malattia infettiva compresa nell'apposito elenco pubblicato dal Ministero della Sanità, il Comune deve darne informazione immediatamente alla A.S.L. dove è avvenuto il decesso.

3. Nel caso di decesso senza assistenza medica la denuncia della presunta causa di morte è fatta dal medico necroscopo.

4. I medici incaricati di eseguire autopsie disposte dall'autorità giudiziaria o per riscontro diagnostico sono ugualmente tenuti all'obbligo di denunciare la causa della morte, osservate le disposizioni contenute negli artt. 39 e 45 del D.P.R. 10.9.1990, n. 285.

5. In tutti i casi previsti dai commi precedenti la denuncia della causa di morte deve essere fatta entro ventiquattro ore dall'accertamento del decesso su apposita sche

Sei un esperto in grado di generare domande a risposta chiusa su un regolamento comunale, per una persona la quale deve sostenere un esame di ammissione per una selezione a scopo assunzione.
L'esame richiede la conoscenza degli articoli del regolamento.  Ogni articolo ha un titolo ed un testo.
Dato un articolo, genera un risultato composto da:
. una domanda 
. possibili risposte a tale domanda, in cui la prima risposta è giusta, le altre sono sbagliate.  
. la spiegazione del perchè la prima risposta è vera, considerando il testo dell'articolo.
Le risposte sbagliate contengono informazioni plausibili ma in contrasto o non consistenti con quanto scritto nell'articolo.

Il risultato deve essere formattato in JSon, come di seguito.
Il campo question contiene la domanda, il campo answers contiene la lista delle possibili risposte.
Genera sempre tre possibili risposte alla domanda.

Esempio del formato della risposta:

{
  "question": "Domanda",
  "answers": ["Risposta 1", "Risposta 2", "Risposta 3"],
  "explanation": "Spiegazione del perche' la prima risposta è vera."
}


Nel formulare la domanda e le risposte, utilizza solamente il titolo e testo dell'articolo, non introdurre altre informazioni estranee ad esso o inventate. Genera solamente Json valido.

Esempio di articolo:
Titolo: UBICAZIONE E ORARI DEI CIMITERI

Testo: Il Comune ha inoltre Cimiteri Speciali nelle seguenti frazioni: - Casale Popolo, Terranova, Santa Maria del Tempio, San Germano, Torcello-Rolasco, Roncaglia.  L'orario di apertura al pubblico dei cimiteri viene determinato dal Sindaco a seconda delle stagioni. Nei giorni di neve il cimitero resta chiuso ai visitatori, sino a quando si saranno praticati gli opportuni passaggi ed accessi.
 
Esempio di risposta:
{
  "question": "Chi decide l'orario di apertura al pubblico dei cimiteri?",
  "answers": ["Il Sindaco, inoltre l'orario dipende dalle stagioni", "Il consiglio comunale ad ogni inizio anno, riunito in seduta plenaria", "L'assessore ai lavori pubblici, tramite referendum che coinvolge i cittadini"],
  "explanation":"L'articolo specifica che l'orario di apertura al pubblico dei cimiteri viene determinato dal Sindaco a seconda delle stagioni."
}

Di seguito l'articolo su cui devi generare il risultato:
---

Causa di Morte
1. A norma dell'art. 103 T.U. Leggi Sanitarie, R.D. 27.7.1934 n. 1265, i medici debbono per ogni caso di morte di persona da loro assistita denunciare al Sindaco la malattia che, a loro giudizio, ne sarebbe stata la causa. 

2. Nel caso di morte per malattia infettiva compresa nell'apposito elenco pubblicato dal Ministero della Sanità, il Comune deve darne informazione immediatamente alla A.S.L. dove è avvenuto il decesso.

3. Nel caso di decesso senza assistenza medica la denuncia della presunta causa di morte è fatta dal medico necroscopo.

4. I medici incaricati di eseguire autopsie disposte dall'autorità giudiziaria o per riscontro diagnostico sono ugualmente tenuti all'obbligo di denunciare la causa della morte, osservate le disposizioni contenute negli artt. 39 e 45 del D.P.R. 10.9.1990, n. 285.

5. In tutti i casi previsti dai commi precedenti la denuncia della causa di morte deve essere fatta entro ventiquattro ore dall'accertamento del decesso su apposita scheda di morte stabilita dal Ministero della Sanità d'intesa con l'Istituto Nazionale di Statistica.

6. Fermo restando per i sanitari l'obbligo di cui all'art. 365 del codice penale, ove dalla scheda di morte risulti o sorga comunque il sospetto che la morte sia dovuta a reato, il Sindaco deve darne immediata comunicazione all’autorità giudiziaria ed a quella di pubblica sicurezza.

In [18]:
promptTemplate="""
Sei un esperto in grado di generare domande a risposta chiusa su un regolamento comunale, per una persona la quale deve sostenere un esame di ammissione per una selezione a scopo assunzione.
L'esame richiede la conoscenza degli articoli del regolamento.  Ogni articolo ha un titolo ed un testo.
Dato un articolo, genera un risultato composto da:
. una domanda 
. possibili risposte a tale domanda, in cui la prima risposta è giusta, le altre sono sbagliate.  
. la spiegazione del perchè la prima risposta è vera, considerando il testo dell'articolo.
Le risposte sbagliate contengono informazioni plausibili ma in contrasto o non consistenti con quanto scritto nell'articolo.

Il risultato deve essere formattato in JSon, come di seguito.
Il campo question contiene la domanda, il campo answers contiene la lista delle possibili risposte.
Genera sempre tre possibili risposte alla domanda.

Esempio del formato della risposta:

{
  "question": "Domanda",
  "answers": ["Risposta 1", "Risposta 2", "Risposta 3"],
  "explanation": "Spiegazione del perche' la prima risposta è vera."
}


Nel formulare la domanda e le risposte, utilizza solamente il titolo e testo dell'articolo, non introdurre altre informazioni estranee ad esso o inventate. Genera solamente Json valido.

Esempio di articolo:
TITOLO: 
UBICAZIONE E ORARI DEI CIMITERI

TESTO: 
Il Comune ha inoltre Cimiteri Speciali nelle seguenti frazioni: - Casale Popolo, Terranova, Santa Maria del Tempio, San Germano, Torcello-Rolasco, Roncaglia.  L'orario di apertura al pubblico dei cimiteri viene determinato dal Sindaco a seconda delle stagioni. Nei giorni di neve il cimitero resta chiuso ai visitatori, sino a quando si saranno praticati gli opportuni passaggi ed accessi.
 
Esempio di risposta:
{
  "question": "Chi decide l'orario di apertura al pubblico dei cimiteri?",
  "answers": ["Il Sindaco, inoltre l'orario dipende dalle stagioni", "Il consiglio comunale ad ogni inizio anno, riunito in seduta plenaria", "L'assessore ai lavori pubblici, tramite referendum che coinvolge i cittadini"],
  "explanation":"L'articolo specifica che l'orario di apertura al pubblico dei cimiteri viene determinato dal Sindaco a seconda delle stagioni."
}

Di seguito l'articolo su cui devi generare il risultato:
---
"""

In [19]:
if document.numArticles>0 :
    indxArticle = random.randint(0, document.numArticles-1)
    article = document.article(indxArticle)

    arguments= {'title':article.title, 'text':article.text}
    prompt= copy.copy(promptTemplate)
    prompt += f'TITOLO: { article.title}\n\n' 
    prompt += f'TESTO:  { article.text}\n' 
    print(prompt)


Sei un esperto in grado di generare domande a risposta chiusa su un regolamento comunale, per una persona la quale deve sostenere un esame di ammissione per una selezione a scopo assunzione.
L'esame richiede la conoscenza degli articoli del regolamento.  Ogni articolo ha un titolo ed un testo.
Dato un articolo, genera un risultato composto da:
. una domanda 
. possibili risposte a tale domanda, in cui la prima risposta è giusta, le altre sono sbagliate.  
. la spiegazione del perchè la prima risposta è vera, considerando il testo dell'articolo.
Le risposte sbagliate contengono informazioni plausibili ma in contrasto o non consistenti con quanto scritto nell'articolo.

Il risultato deve essere formattato in JSon, come di seguito.
Il campo question contiene la domanda, il campo answers contiene la lista delle possibili risposte.
Genera sempre tre possibili risposte alla domanda.

Esempio del formato della risposta:

{
  "question": "Domanda",
  "answers": ["Risposta 1", "Risposta 2", "R

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
modelName="gemini-2.0-flash"

maxOutputTokens=8192
temperature=1.0

llm = ChatGoogleGenerativeAI(model=modelName, 
                             max_output_tokens=maxOutputTokens, 
                             temperature=temperature)

if document.numArticles>0 :
    indxArticle = random.randint(0, document.numArticles-1)
    article = document.article(indxArticle)

    arguments= {'title':article.title, 'text':article.text}
    prompt= copy.copy(promptTemplate)
    prompt += f'TITOLO: { article.title}\n\n' 
    prompt += f'TESTO:  { article.text}\n' 
    result = llm.invoke(prompt)
    # pprint.pp(result.content)
    
    # print(result)
    textContent= str(result.content)
    if textContent.startswith("```json"):        
        textContent= textContent.replace("```json", "", 1)
    if textContent.endswith("```"):
        textContent= textContent[0:-3]
    # print(textContent)    
    try:
        parsedResult = json.loads( textContent)
        pprint.pp(parsedResult)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")


{'question': 'Cosa accade ad una salma non mineralizzata al momento '
             "dell'esumazione o estumulazione per scadenza della concessione?",
 'answers': ['Viene inumata nel campo comune, con la provvista di un eventuale '
             'altro feretro a carico degli interessati.',
             "Viene cremata a spese del comune e le ceneri disperse in un'area "
             'apposita.',
             'Viene tumulata nuovamente nello stesso loculo per un periodo '
             'massimo di 5 anni, a spese del comune.'],
 'explanation': "Il comma 1 dell'articolo specifica che se la salma non è "
                "ridotta allo stato osseo al momento dell'esumazione o "
                "dell'estumulazione per scadenza della concessione, essa sarà "
                'inumata nel campo comune, con la provvista di un eventuale '
                'altro feretro a carico degli interessati.'}
